In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import cmath
import os

def plot_img(fig):
  plt.figure(figsize=(10,10))
  plt.imshow(fig, cmap='gray', vmin=0, vmax=255)
  plt.axis('off')
  plt.show()


def fft_padding(img, kernel):
  p = img.shape[0] + kernel.shape[0] - 1
  q = img.shape[1] + kernel.shape[1] - 1
  new_img = np.zeros((p, q))
  new_kernel = np.zeros((p, q))
  for x in range(img.shape[0]):
    for y in range(img.shape[1]):
      new_img[x][y] = img[x][y]
  for x in range(kernel.shape[0]):
    for y in range(kernel.shape[1]):
      new_kernel[x][y] = kernel[x][y]
  return new_img, new_kernel


file_list = []
for file_path in os.listdir('/content'):
    # check if current file_path is a file
    if os.path.isfile(os.path.join('/content', file_path)):
        # add filename to list
        file_list.append(file_path)

folder_path = '/content/processed_images'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for file_name in file_list:
  img = cv2.imread(file_name, 0)
  plot_img(img)
  blur = cv2.GaussianBlur(img,(5,5),0)
  th1 = cv2.adaptiveThreshold(blur.astype('uint8'),255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
 cv2.THRESH_BINARY_INV,11,2)
  plot_img(th1)
  contours, hierarchy = cv2.findContours(th1.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  cv2.drawContours(img, contours, -1, (0,255,0), cv2.FILLED)
  plot_img(img)
  img = cv2.medianBlur(img, 5)
  plot_img(img)
  sobel_kernel_x = np.array([[-1, 0, 1],
                           [-2, 0, 2],
                           [-1, 0, 1]])
  padded_img, padded_sobel_x = fft_padding(img, sobel_kernel_x)


  sobel_kernel_y = np.array([[-1, -2, -1],
                           [0, 0, 0],
                           [1, 2, 1]])
  padded_img, padded_sobel_y = fft_padding(img, sobel_kernel_y)

  fft_img = np.fft.fft2(padded_img)
  fft_sobel_x = np.fft.fft2(padded_sobel_x)
  fft_sobel_y = np.fft.fft2(padded_sobel_y)
  fft_img_x = fft_img * fft_sobel_x
  fft_img_y = fft_img * fft_sobel_y

  img_x = np.round(np.fft.ifft2(fft_img_x).real)
  img_y = np.round(np.fft.ifft2(fft_img_y).real)

  filtered_img = (img_x ** 2 + img_y ** 2) ** (1/2)
  plot_img(filtered_img)
  cv2.imwrite(f'{folder_path}/processed_{file_name}.tif', filtered_img)


In [ ]:
!zip -r /content/processed_images.zip /content/processed_images
from google.colab import files
files.download('/content/processed_images.zip')